# Virtual Paint

**Declaration** - Unfortunately, openCV isn't completely supported on Google Colab, so it will crash. Also, you are gonna need a good webcam to detect objects clearly. If you'd like to try this, best option is to run it locally. Only packages you are gonna need are - opencv-python, numpy.

## Importing required libraries

In [1]:
import cv2
import numpy as np

## Setting up Camera Parameters

In [2]:
frameWidth = 640
frameHeight = 480

cap = cv2.VideoCapture(0)
print(f"Camera Status - {cap.isOpened()}")

cap.set(3, frameWidth)  # width
cap.set(4, frameHeight) # height
cap.set(10,150)  # brightness

Camera Status - True


False

## List of valid colors
Colors that our virtual paint can detect, in HSV format.

To add more custom colors, go [here](https://alloyui.com/examples/color-picker/hsv.html). Also, [this](https://answers.opencv.org/question/134248/how-to-define-the-lower-and-upper-range-of-a-color/) might help.

In [3]:
colors = {
    "orange": {
        "hsv_lower":[5, 107, 0],
        "hsv_upper":[19, 255, 255],
    },
    "purple": {
        "hsv_lower":[133, 56, 0],
        "hsv_upper":[159, 156, 255],
    },
    "green": {
        "hsv_lower":[57, 76, 0],
        "hsv_upper":[100, 255, 255],
    }
}

# list of colors in BGR format to draw on image
drawingColors = {
    "orange": [51,153,255],
    "purple": [255,0,255],
    "green": [0,255,0],
}


## Find colors

In [4]:
def findColor(img, colors, drawingColors):
    imgHSV = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    newPoints = []
    for color in colors.keys():
        lower = np.array(colors[color]["hsv_lower"])
        upper = np.array(colors[color]["hsv_upper"])
        mask = cv2.inRange(imgHSV, lower, upper)
        # show mask
        cv2.imshow(f"{color}", mask)
        
        x, y = getContours(mask)
        cv2.circle(imgResult, (x, y), 10, drawingColors[color], cv2.FILLED)
        
        if x!=0 and y!=0:
            newPoints.append([x, y, color])
    return newPoints

## Get Contours

In [5]:
def getContours(img):
    contours, hierarchy = cv2.findContours(
        img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE
    )
    x, y, w, h = 0, 0, 0, 0
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > 500:
            # check bounding box
            # cv2.drawContours(imgResult, cnt, -1, (255, 0, 0), 3)
            
            peri = cv2.arcLength(cnt, True)
            approx = cv2.approxPolyDP(cnt, 0.02 * peri, True)
            x, y, w, h = cv2.boundingRect(approx)
    return x + w // 2, y

## Draw on canvas

In [6]:
def drawOnCanvas(myPoints, drawingColors):
    for x, y, color in myPoints:
        cv2.circle(imgResult, (x, y), 10, drawingColors[color], cv2.FILLED)

## Execution

In [8]:
# list to store points
myPoints = []

while cap.isOpened():
    success, img = cap.read()
    imgResult = img.copy()

    newPoints = findColor(img, colors, drawingColors)

    # add new points to list
    if len(newPoints) != 0:
        myPoints.extend(newPoints)

    if len(myPoints) != 0:
        drawOnCanvas(myPoints, drawingColors)

    cv2.imshow("Result", imgResult)
    
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
        
cap.release()
cv2.destroyAllWindows()